In [46]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os
import time

In [47]:
def db_save(NEWS_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            NEWS_LIST.to_sql(name = 'NEWS_LIST', con = con, index = False, if_exists='append')
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(NEWS_LIST), '건 저장완료..')

In [48]:
def db_delete():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            cur = con.cursor()
            sql = 'DELETE FROM NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e))

In [49]:
def db_select():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            query = 'SELECT * FROM NEWS_LIST'
            NEWS_LIST = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e))
        return NEWS_LIST


In [50]:
import re
import string

def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·‘’'']'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@class="article_view"]/section/p'):
        if p.text: # 체크
            body.append(re.sub(punc, '', p.text)) # 특수문자 제거
    full_body = ' '.join(body)

    return full_body

In [51]:
page = 74
max_page = 0
REG_DATE = '20230820'

In [52]:
while(True):
    df_list = []
    response = requests.get(f'http://news.daum.net/breakingnews/digital?page={page}&regDate={REG_DATE}')
    root = lxml.html.fromstring(response.content)
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a = li.xpath('div/strong/a')[0]
        url = a.get('href')
        article = get_detail(url)
        df_list.append([url, a.text, article])

    if df_list:
        df_10 = pd.DataFrame(df_list)
        df_10.columns = ['URL', 'TITLE', 'ARTICLE']
        db_save(df_10)

    # 페이지 번호 중에서 max 페이지 가져오기
    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
        try:
            num = int(a.text)
            if max_page < num:
                max_page = num
        except:
            pass

    # 마지막 페이지 여부 확인
    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@class="btn_page btn_next"]')

    if (len(span) <= 0) & (page >= max_page):
        break
    else:
        page = page + 1

    time.sleep(1)

6 건 저장완료..


In [56]:
# db_delete()

In [54]:
news_df = db_select()
print(news_df)

                                        URL  \
0    https://v.daum.net/v/20230809095212544   
1    https://v.daum.net/v/20230809095157531   
2    https://v.daum.net/v/20230809095155529   
3    https://v.daum.net/v/20230809095139520   
4    https://v.daum.net/v/20230809095120511   
..                                      ...   
589  https://v.daum.net/v/20230820060118890   
590  https://v.daum.net/v/20230820055005778   
591  https://v.daum.net/v/20230820053201722   
592  https://v.daum.net/v/20230820050041630   
593  https://v.daum.net/v/20230820045214586   

                                                 TITLE  \
0                                   비만인 사람의 뇌, 이것이 달랐다   
1                 [컨콜] 엔씨 "콘솔 신작 프로젝트M, 아이온2 이후 출시 예상"   
2                            인프랩, '인프런'으로 IT 배움 장벽 낮춘다   
3                    바디텍메드, 2분기 영업이익 82억원…전년대비 168% 증가   
4                          펄펄 끓는 지구…'탄소 배출'이 원인이 아니라고?   
..                                                 ...   
589  [인터뷰] ‘인공지능 석학’ 제리 카플란 “

In [55]:
excel = pd.ExcelWriter('news.xlsx')
news_df.to_excel(excel, 'Daum뉴스', index=False)
excel.save()

<ipython-input-55-298052c84b40>:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel.save()
